Import Libraries and necessary datasets

In [96]:
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt

from google.colab import drive
drive.mount("/content/drive/")

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [97]:
!ls /content/drive/MyDrive/DSBDALab/Lab5/Datasets/

air_quality.csv  heart.csv


In [98]:
AQ_df = pd.read_csv("/content/drive/MyDrive/DSBDALab/Lab5/Datasets/air_quality.csv", encoding='Windows-1252')
AQ_df.head()

<ipython-input-98-06efdb386281>:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  AQ_df = pd.read_csv("/content/drive/MyDrive/DSBDALab/Lab5/Datasets/air_quality.csv", encoding='Windows-1252')


,stn_code,sampling_date,state,location,agency,type,so2,no2,rspm,spm,location_monitoring_station,pm2_5,date
0,150.0,February - M021990,Andhra Pradesh,Hyderabad,NaN,"Residential, Rural and other Areas",4.8,17.4,NaN,NaN,NaN,NaN,1990-02-01
1,151.0,February - M021990,Andhra Pradesh,Hyderabad,NaN,Industrial Area,3.1,7.0,NaN,NaN,NaN,NaN,1990-02-01
2,152.0,February - M021990,Andhra Pradesh,Hyderabad,NaN,"Residential, Rural and other Areas",6.2,28.5,NaN,NaN,NaN,NaN,1990-02-01
3,150.0,March - M031990,Andhra Pradesh,Hyderabad,NaN,"Residential, Rural and other Areas",6.3,14.7,NaN,NaN,NaN,NaN,1990-03-01
4,151.0,March - M031990,Andhra Pradesh,Hyderabad,NaN,Industrial Area,4.7,7.5,NaN,NaN,NaN,NaN,1990-03-01


In [99]:
heart_df = pd.read_csv("/content/drive/MyDrive/DSBDALab/Lab5/Datasets/heart.csv")
heart_df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,52,1,0,125,212,0,1,168,0,1.0,2,2,3,0
1,53,1,0,140,203,1,0,155,1,3.1,0,0,3,0
2,70,1,0,145,174,0,1,125,1,2.6,0,0,3,0
3,61,1,0,148,203,0,1,161,0,0.0,2,1,3,0
4,62,0,0,138,294,1,1,106,0,1.9,1,3,2,0


In [100]:
heart_df.columns

Index(['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach',
       'exang', 'oldpeak', 'slope', 'ca', 'thal', 'target'],
      dtype='object')

Data Cleaning

In [101]:
heart_df.isnull().sum()

age         0
sex         0
cp          0
trestbps    0
chol        0
fbs         0
restecg     0
thalach     0
exang       0
oldpeak     0
slope       0
ca          0
thal        0
target      0
dtype: int64

In [102]:
AQ_df.dropna(thresh=0.5*len(AQ_df), axis=1, inplace=True)
heart_df.drop_duplicates(inplace=True)

In [103]:
heart_df.isnull().sum()

age         0
sex         0
cp          0
trestbps    0
chol        0
fbs         0
restecg     0
thalach     0
exang       0
oldpeak     0
slope       0
ca          0
thal        0
target      0
dtype: int64

In [104]:
heart_df.dropna(thresh=0.5*len(heart_df), axis=1, inplace=True)

In [105]:
heart_df.duplicated().sum()

0

In [106]:
heart_df.drop_duplicates(inplace=True)

Data Merge

In [107]:
spli11 = heart_df[['age' , 'chol']]
split2 = heart_df[['age' , 'trestbps']]

In [108]:
merged = pd.merge(spli11 , split2 , on="age", how="inner")
merged.head()

,age,chol,trestbps
0,52,212,125
1,52,212,128
2,52,212,134
3,52,212,118
4,52,212,138


Data Transformation

In [110]:
heart_df['target'] = heart_df['target'].apply(lambda a:1 if a>0 else 0)

Data Model Building

In [111]:
from sklearn.model_selection import train_test_split
X = heart_df.drop('target' ,axis = 1)
Y = heart_df.target

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

In [112]:
from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression()
logreg.fit(X_train , Y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

Evaluating Data Model

In [113]:
from sklearn.metrics import classification_report, confusion_matrix
y_pred = logreg.predict(X_test)

print(confusion_matrix(Y_test, y_pred))
print(classification_report(Y_test, y_pred))

[[37 11]
 [ 4 39]]
              precision    recall  f1-score   support

           0       0.90      0.77      0.83        48
           1       0.78      0.91      0.84        43

    accuracy                           0.84        91
   macro avg       0.84      0.84      0.84        91
weighted avg       0.84      0.84      0.83        91

